In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random
from collections import Counter
from collections import OrderedDict
# from ipynb.fs.full.dev_notebook import stringToQiskitSingleGate

from generator import generateCliffordCircuit

In [3]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [4]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['Z', 'Z', 'I', 'H', 'X'], ['CNOT_T', 'I', 'I', 'CNOT_C', 'I'], ['X', 'I', 'H', 'I', 'Z'], ['I', 'I', 'I', 'CNOT_T', 'CNOT_C'], ['X', 'S', 'H', 'H', 'H']]
     ┌───┐ ░ ┌───┐ ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ Z ├─░─┤ X ├─░─┤ X ├─░───────░─┤ X ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤
q_1: ┤ Z ├─░───┼───░─┤ I ├─░───────░─┤ S ├
     ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤
q_2: ┤ I ├─░───┼───░─┤ H ├─░───────░─┤ H ├
     ├───┤ ░   │   ░ ├───┤ ░ ┌───┐ ░ ├───┤
q_3: ┤ H ├─░───■───░─┤ I ├─░─┤ X ├─░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_4: ┤ X ├─░───────░─┤ Z ├─░───■───░─┤ H ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [5]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")

In [6]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('YYXYX')

In [68]:
def params_list(operation_set, pauli_set=["X","Y","Z","I"]):
    '''This function generates a dictionary of parameters for the given operation set.
    Inputs: operation_set - A list of operations.
            pauli_set - A list of pauli operators.
    Outputs: params - A dictionary of parameters, with each set to 0.'''

    params = OrderedDict()
    for operation in operation_set:
        if operation == "CNOT_C" or operation == "CNOT_T":
            print("reached here")
            for pauli1 in pauli_set:
                for pauli2 in pauli_set:
                    pauli = pauli1+pauli2
                    params[("CNOT", pauli)] = 0
        else:
            print("reached here 2")
            for pauli in pauli_set:
                params[(operation, pauli)] = 0
    return params

print(params_list(singleGateSet))

reached here 2
reached here 2
reached here 2
reached here 2
reached here 2
OrderedDict([(('X', 'X'), 0), (('X', 'Y'), 0), (('X', 'Z'), 0), (('X', 'I'), 0), (('H', 'X'), 0), (('H', 'Y'), 0), (('H', 'Z'), 0), (('H', 'I'), 0), (('Z', 'X'), 0), (('Z', 'Y'), 0), (('Z', 'Z'), 0), (('Z', 'I'), 0), (('I', 'X'), 0), (('I', 'Y'), 0), (('I', 'Z'), 0), (('I', 'I'), 0), (('S', 'X'), 0), (('S', 'Y'), 0), (('S', 'Z'), 0), (('S', 'I'), 0)])


In [69]:
def remove_negatives(pauli):
    '''This function takes a pauli operator and removes the negative sign from it.'''

    if pauli.to_label()[0] == "-":
        return qk.quantum_info.Pauli(pauli.to_label()[1:])
    else:
        return pauli

In [71]:
def pauli_hist(circuit, pauli_initial, params_list=params_list(singleGateSet)):
    '''This function computes the histogram of pauli operators after the circuit has been applied.
    Inputs: circuit - A Qiskit Circuit, Clifford, or Gate object.
            pauli_initial - A Qiskit Pauli object.
            params_list - A dictionary of parameters for the circuit.
    Outputs: pauli_counts - A dictionary of pauli operators and their counts.'''

    pauli = pauli_initial

    for op_instructions in circuit.data:
        if op_instructions.operation.name.upper() == "ID" or op_instructions.operation.name.upper() == "I" or op_instructions.operation.name.upper() == "BARRIER":
            print("skipped ", op_instructions.operation.name.upper(), " operation")
            break
        elif op_instructions.operation.name.upper() == "CX" or op_instructions.operation.name.upper() == "CNOT":
            params_list[("CNOT_C", pauli.to_label()[0])] += 1
            params_list[("CNOT_T", pauli.to_label()[1])] += 1
            pauli = qk.quantum_info.Pauli(pauli.to_label()[1] + pauli.to_label()[0] + pauli.to_label()[2:])
        else:
            pauli = remove_negatives(pauli)
            params_list[(op_instructions.operation.name.upper(), pauli.to_label())] += 1
            pauli = Clifford_Permute(op_instructions.operation, pauli)

    return params_list

print(pauli_hist(qiskitCircuit, qk.quantum_info.Pauli('X')))

reached here 2
reached here 2
reached here 2
reached here 2
reached here 2
skipped  BARRIER  operation
OrderedDict([(('X', 'X'), 0), (('X', 'Y'), 0), (('X', 'Z'), 0), (('X', 'I'), 0), (('H', 'X'), 0), (('H', 'Y'), 1), (('H', 'Z'), 0), (('H', 'I'), 0), (('Z', 'X'), 1), (('Z', 'Y'), 1), (('Z', 'Z'), 0), (('Z', 'I'), 0), (('I', 'X'), 0), (('I', 'Y'), 0), (('I', 'Z'), 0), (('I', 'I'), 0), (('S', 'X'), 1), (('S', 'Y'), 1), (('S', 'Z'), 0), (('S', 'I'), 0)])


In [72]:
def generate_first_layer(length, pauli_set=["X","Y","Z","I"]):
    first_layer = ""
    for ele in np.random.choice(pauli_set, length):
        first_layer += ele
    return first_layer.upper()

print(generate_first_layer(5))

XIXXI


In [78]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

qiskitCircuit = transpileListToQiskitCircuit(generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet))
print(qiskitCircuit)
stringfirstlayer = generate_first_layer(qiskitCircuit.num_qubits)
first_row = pauli_hist(qiskitCircuit, qk.quantum_info.Pauli(stringfirstlayer))

     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ X ├─░───────░─┤ X ├─░───────░─┤ S ├
     ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤
q_1: ┤ H ├─░─┤ X ├─░─┤ X ├─░───────░─┤ I ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤
q_2: ┤ X ├─░───■───░─┤ X ├─░───────░─┤ S ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_3: ┤ X ├─░───────░─┤ Z ├─░───■───░─┤ I ├
     ├───┤ ░       ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_4: ┤ Z ├─░───────░─┤ Z ├─░─┤ X ├─░─┤ I ├
     └───┘ ░       ░ └───┘ ░ └───┘ ░ └───┘


KeyError: ('X', 'IXIIZ')

In [ ]:
def generate_row():
    '''This function generates a row of the circuit, which is a dictionary of pauli operators and their counts.'''

    row = OrderedDict()
    for key in first_row:
        row[key] = np.random.randint(0, 4)
    return row

In [86]:
def generate_A(length=5):
    '''This function generates the A matrix by randomly generating each row, checking if it increases the rank and then continuing onwards
    Inputs: numrow - The number of rows in the A matrix.
            numcol - The number of columns in the A matrix.
    Outputs: A - The A matrix.'''

    A = np.zeros((length, length))
    while np.linalg.matrix_rank(A) < A.shape[1]:
        new_row = generate_row_A()
        A_star = np.vstack([A, new_row])
        if np.linalg.matrix_rank(A_star) > np.linalg.matrix_rank(A):
            A = A_star
    return A

print(generate_A(5))


NameError: name 'generate_row_A' is not defined